# 以图生图
- 图像分割
- 风格转换
- 其他

# 图像分割Image Segmentation

> 分割, 分类, 检测是cv的3个基本问题

## 三者比较:

#### 分类: 

输入图片

输出类别向量或标量

3维或2维变成1维或0维

可以看成语义提取

#### 检测:
位置回归, 类别分类

输入图片

输出位置向量和类别向量

#### 分割:
以图生图



输入图, 输出还是图, 这个图要对原图的每一个像素的类别进行判别



1. 每个像素都对应, 宽高大小不变, 每个channel都是一个类别, 每个像素只有0,1是还不是这种信息. 相当于多个2分类

2. 每个像素都对应, 宽高大小不变, 但只有1个channel, 每个像素的数值可以不固定 

**子领域:**
- 语义分割semantic
    - 树, 
    - 蓝天白云, 都算背景
    - 多个人同一类
- 实例分割instance
    - 多个人不是同一类
- 全景分割panoptic
    - 结合前两个



## 分割的难点
- 多尺度
- 多姿态(多视角)
- 物体之间遮挡
- 光照
    - 分割在医疗影响, 无人车, 航拍, 机器人方向应用很多
- 边缘
    - 比如人, 分割要描出轮廓
    - 如果是仙人掌,就很难
- 人工标注昂贵有噪声
- 尺寸不变导致网络更需要显存,实时性难
- 语义交互, 先验知识的利用
    - 笔记本总是和桌子交互在一起
    - 交通灯总是在路中间



# 分割应用

![1](1.PNG)

航拍图, Label是人工标注, FCN最好

![2](2.PNG)

医疗

![3](3.PNG)

无人车

## 四种网络
- FCN全卷积网络
- UNet/ENet
- Mask/RCNN
- Developments

## FCN网络 2015

[论文地址](https://people.eecs.berkeley.edu/%7Ejonlong/long_shelhamer_fcn.pdf)

![4](4.PNG)

1. 经过5次pooling, 长和高都是原来的1/32

2. 第一种方案：然后做上采样(小图变大图)，直接变回原尺寸

> 上采样的方式:
- 插值
- 转置卷积(同义词: 反卷积/分数步长卷积Fractional stride convolution)
    - X是4x4的矩阵, 经过卷积变成一个2x2的Y
    - 很可能经过了卷积: 3x3, stride=1,padding=0
    - 那么可以写成 Y(4x1) = C(4x16) · X(16x1)
    - 反过来X(16x1) = C^T(16x4) · Y(4x1)
    - C^T未必是C^T
    - 需要先在小图周围做很多padding0，再看看上采样的参数（步长，倍率）然后插值很多0

3. (第二种方案)为了保留位置信息和语义信息，把不同的尺度进行融合，把 上次的pooling的结果 和 这次pooling结果的2x 做elementwise的相加。然后再上采样得到原图1/2大小

4. 把多个pooling的结果转化成同一尺度再elementwise的相加， 然后做原图1/4的上采样


### 特点：
- 没有FC，没有向量，直接大图变小图，小图再变大图
- 上采样需要学习细节， 用了转置卷积。转置卷积是可以学习的
- 深层浅层的特征融合Shortcut between lower and deeper layers

为什么pooling1和pooling2不融合呢？作者实践发现效果不够好



## Unet 2015

[论文地址](https://arxiv.org/pdf/1505.04597.pdf)

![5](5.jpg)


![6](6.PNG)

白色和蓝色只是channel维度上连接在一起，不是elementwise相加


声纳，声波，医疗用的多

其实原图就是388x388x2，先padding到了572, 为什么？

原因：医学领域每张图片 来之不易，图像边缘也有信息：

Tile Strategy：

padding填充图片边缘本身（镜像padding）


为什么只在同尺度融合？也就是只在平行的地方融合？

效果一般。

（面试重点）




Unet很准，但是很慢，医疗可以，无人车就算了

## 改进速度：E-Net
[论文地址](https://arxiv.org/pdf/1606.02147.pdf)

![7](7.PNG)

卷积层数增加，特征图尺寸减小，但感受野增加，通道数增加，信息并不损失，所以大多数都是有大到小再由小到大

为什么ENet中间为什么尺度不变：
- 为了加速，通道数少，只有128
- 再缩小尺寸对速度不明显
- 为了增加感受野，用空洞卷积

加速手段：
- 通道数减少
- 尺寸不变
- 非对称卷积 

此外借鉴了ResNet的BottleNeck模块，而且微调了

## Mask-RCNN 2017

[论文地址](https://arxiv.org/pdf/1703.06870.pdf)

使用了FPN作为backbone

精度高, 尤其是重叠时, 5fps

## Developments

- FCN
    - 深度学习领域(不算传统cv)比较早的
- 转置卷积->插值的趋势
- DeepLab: FCN和CRF(条件向量场)结合的
- 空洞卷积是个趋势
- BackBone不断改进, 
    - VGG,ResNet, EfficientNet,Repnet2020
- 多尺度融合的趋势(金字塔)
- 正负样本划分: 
    - ICNet(30FPS),级联, 每次选的正负样本不一样,
- 针对标注困难: 半监督,非监督,自监督
    - 自监督是只有数据没有label,但是前预设一些伪标签
    - [论文](https://arxiv.org/pdf/1502.02734.pdf)

# 风格转换 


> 把一张艺术照风格变换到实际照片上

## [2015论文](https://arxiv.org/pdf/1603.08155.pdf)观点:
- 任意一张图片都可以表示为内容+风格
- 层数越深, 语义信息多, 局部信息(位置,边缘,纹理)少, 反之亦然
- 图A的内容Ac, 风格As, 图B的内容Bc, 风格Bs, 我们需要一张图X, 使得L(Xs-As)+L(Xc-Bc)最小

### 过程:

![8](8.PNG)

1. 风格图经过Vgg16,长高缩小1/16,(去掉最后一次pooling)
2. 每一层不同深度的特征图出来,转换成Gram矩阵, 最小的叫AL, 然后AL-1等等
3. 内容图片也用Vgg16, 但是参数不训练, 用别人训练好的
4. 一张噪声图,作为求解的初始值,也输入vgg16,得到特征图,每一层与风格图的特征图做加权Loss
5. 求解图的特征图, 只有最第4层和内容图的第4层做逐像素点的L2 loss


### 为什么内容只用一层的Loss,而风格每层都用?

比如拍照, 位置偏移一点, 内容其实是不变的,但是这两张L2loss很大

我们需要保持平移不变性, 对深层语义信息提取

也就是内容是空间无关的



### Gram矩阵

什么是风格

图片中某些纹理, 比如三个固定相对距离的蓝点总是重复出现

Gram矩阵是对特征相关性的一种描述

两种算法:

1. HxWxC的特征图

每个channel拿出来, 每个通道表示原图的一种响应

协方差矩阵:CxC对称方阵

每个i,j位置的元素都是第i个特征图和第j个特征图的 element相乘再相加 可以再除以WxHxC做一个归一化

2. HxWxC的特征图

每个2维特征图变成一维向量, 得到c行, 这个矩阵乘以它的转置, 和上面的得到的Gram矩阵一样的

但是2015这个文章每次都要训练, 比较慢. 好处是不用训练集



## [2016论文](https://cs.stanford.edu/people/jcjohns/papers/eccv16/JohnsonECCV16.pdf)

- 把问题模块化
- 使用时更快, 不用重复训练
- Perceptual Loss 感知Loss
- L1适合细节不适合颜色, L2相反


## 结构&过程

![9](9.PNG)


Image Transform Net需要训练

输出一个图

再进入LossNet(不用训练), 分别计算风格Loss和内容Loss


## 感知Loss

![10](10.PNG)

上采样时, 生成图片往往有棋盘格的奇怪纹理

这是由于转置卷积本身稀疏导致的

为了约束这种纹理, 加了一种Loss,限制梯度

这里就是
$$L_{TV}=\sqrt{G_{x}^{2}+G_{y}^{2}}$$


##  特征模仿Feature Mimicking/模型蒸馏Model Distillation

模型蒸馏和特征模仿有一定的类似

为了提高速度:

0. 模型的设计, shufflenet,mobilenet等
1. 模型量化, 参数的类型压缩,
2. 删除和剪枝, 小的权重丢弃, 有些神经元不需要
3. 模型蒸馏


### 模型蒸馏

要把一个大模型变成小模型,"蒸馏出知识", 权重就是知识

用蒸馏过后的知识指导小模型的学习

用小模型替代大模型

https://www.cs.toronto.edu/%7Ehinton/absps/distillation.pdf

https://arxiv.org/pdf/1412.6550.pdf

http://openaccess.thecvf.com/content_cvpr_2017/papers/Li_Mimicking_Very_Efficient_CVPR_2017_paper.pdf

主要用在分类

![11](11.PNG)



a 就是正常的大模型进行分类的训练预测

b 用大模型对要预测的图片生成更软的类别

人工标注的时候,用小数(软),而不是01(硬), 知识更丰富

$$ SoftMax : \;\;a_{i}=\frac{e^{z_{i}}}{\sum_{k=1}^{5} e^{z_{k}}}$$

其实e的次幂可以都除以T,当T等于0, softmax很硬,当T = 无穷大, softmax很软

c 把b得到的软标签保存下来, 当作浓缩的知识, 交给新的小模型训练

小模型同时用软硬两种训练

最终用单独用小模型预测


# 其他

## 图像增强,变亮

通道分离, 比如yuv

模块化, 训练一个网络, 将暗图片变成亮图片, 需要两种数据

高频低频分离

## 去反光

训练集拍摄有玻璃和没玻璃的

## 去雾霾化

## 超分辨,更清晰


